# Knitting

Two types of stitches:
* knit stich (legs faced)
* purl stich (head faced)

Each cell contains 2 dependant knots: to legs of northener stich, head of southerner stich.

Any neighbour combinations are possible.


In [1]:
from random import random
import numpy as np
import sympy as sp
from ipywidgets import widgets
import k3d

from myutils import *
from mycurves import *

t, s, u, v, r, a, b, c, d = sp.symbols("t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P0 P1 P2 P3')

In [2]:
from matplotlib.cm import get_cmap
palette = hexpalette(get_cmap('tab10'))

# Energy model

Energy model by [Choi and Lo, 2003]

* $t \in [0,1]$ for half loop, $t = 0.5$ in the middle of leg
* $w$ — loop width
* $c + 2e$ — total loop height with overlapping
* $c$ — distance between knots 
* $t_h$ — fabric thickness
* $d$ — yarn diameter
* $a$ — fuckery

$x(t) = a t^3 - 1.5 a t^2 + 0.5 (a + w) t$

$y(t) = 0.5(c + 2 e)(1 - cos (\pi t))$

$z(t) = 0.5(t_h - d)(1 - cos (2\pi t))$

* $\chi d$ — horizontal loop separation (distance in knot), $1 ≤ \chi ≤ \frac{w}{2d} - 1$
* $t = \beta_1, \beta_2 $ — righmost and leftmost points of knot $x(\beta_1)  - x(\beta_2) = \chi d$, symmetric: $\beta_1 + \beta_2 = 1$
* $t = \beta_3, \beta_4 $ — intersection points $x(\beta_4) = x(\beta_3), y(\beta_4) - y(\beta_3) = c$

$\beta_{1,2} = \frac{1}{2}\left(1 ± \sqrt{\frac{a - 2w}{3a}}\right)$

$e = \frac{c}{2}\left(\frac{1}{cos \pi \beta_1} - 1\right)$

$t_h = d \left( \frac{2}{cos 2 \pi \beta_3 - cos 2 \pi \beta_4} + 1\right)$

In [3]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("a, w, c, e, z")
Q = vector(
    Ka * t**3 - 3 * half * Ka * t**2 + half*(Ka+Kw)*t,
    half * (Kc + 2*Ke)*(1 - sp.cos(sp.pi * t)),
    half * (Kz) * (1 - sp.cos(2*sp.pi*t))
)


In [4]:
# b1, b2, b3, b4 = sp.symbols("b_1, b_2, b_3, b_4")

In [5]:
# edge points
sp.solve(sp.diff(Q[0], t), t)

[(3*a - sqrt(3)*sqrt(a*(a - 2*w)))/(6*a),
 (3*a + sqrt(3)*sqrt(a*(a - 2*w)))/(6*a)]

In [6]:
controls = {
    'cell': widgets.Checkbox(description="cell", value=True),
    'a': widgets.FloatSlider(description="a", value=5, min=0, max=10.0),
    'w': widgets.FloatSlider(description="w", value=1.0, min=0, max=2.0),
    'c': widgets.FloatSlider(description="c", value=1.0, min=0, max=2.0),    
    'e': widgets.FloatSlider(description="e", value=0.5, min=0, max=1.0, step=0.01),
    'z': widgets.FloatSlider(description="z", value=0.25, min=0, max=2.0)
}
plot = k3d.plot(grid_visible=False)
plot += k3d.vectors(*make_grid((-1,1,0.5),(-1,1,0.5)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-1,1,1),(-1,1,1)), color=0x000000, use_head=False, line_width=0.01)
widgets.GridBox([
    widgets.VBox(list(controls.values())),
    plot], 
    layout=widgets.Layout(grid_template_columns="auto auto"))

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


GridBox(children=(VBox(children=(Checkbox(value=True, description='snap'), FloatSlider(value=5.0, description=…

In [12]:
plot_curve1 = k3d.line(vertices=[], color=palette[1], name="head")
plot += plot_curve1
plot_curve2 = k3d.line(vertices=[], color=palette[2], name="legs")
plot += plot_curve2
plot_points1 = k3d.points(positions=[], color=palette[0], opacity=0.5, point_size=0.1, shader='3d')
plot += plot_points1

In [8]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    snap = np.array(vector(-0.5*params['w'], - params['e'], 0))
    
    Q_ = Q.subs(params)
    spline0 = eval_curve(Q_, t, (0, 1, 36))
    # mirror/rotate
    #spline0 = np.concatenate([spline0, np.flip(spline0, 0) * np.array(vector(-1, -1, 1)) + np.array(vector(0.5*params['w'], 2*params['c'], 0))]) 
        
        
    if params['cell']:
        spline0 = spline0 + snap
        spline1 = np.concatenate([spline0, np.flip(spline0,0) * np.array(vector(-1, 1, 1))])  + np.array(vector(0, - params['c'], 0))
        spline2 = np.concatenate([spline0, np.flip(spline0,0) * np.array(vector(-1, 1, 1))])
    else:
        spline1 = spline0 + np.array(vector(0, - params['c'], 0))
        spline2 = spline0
    curve1.vertices = spline1.astype(np.float32)
    curve2.vertices = spline2.astype(np.float32)
    
#     a, w = params['a'], params['w']
#     t1 = (3*a - np.sqrt(3)*np.sqrt(a*(a - 2*w)))/(6*a)
#     t2 = (3*a + np.sqrt(3)*np.sqrt(a*(a - 2*w)))/(6*a)
#     edges = eval(Q_, {t: [t1, t2]})
#     if params['snap']:
#         edges = edges + snap
#     points1.positions = edges
    

In [9]:
redraw()

In [10]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')

# Spline model

Quadratic uniform spline.

In [3]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("Ka, Kw, Kc, Ke, Kz")
#### 4*2 segments, 5 params
# cpoints = np.array([
#     vector(-0.5-Ka,-Kc-1,-Kz),
#     vector(-0.5+Ka,-Kc-1,-Kz),
#     vector(-0.5+Kw,-1+Ke,+Kz),
#     vector( -Kw,-Ke,+Kz),
#     vector( -Ka, Kc,-Kz),
#     vector( +Ka, Kc,-Kz),
# ])

#### 3*2 segments, 4 params
cpoints = np.array([
    vector(-0.5-Kw,-1-Ke,+Kz),
    vector(-0.5,-Kc-1,-Kz),
    vector(-0.5+Kw,-1-Ke,+Kz),
    vector( -Kw,Ke,+Kz),
    vector(  0, Kc,-Kz),
    vector( +Kw,Ke,+Kz),
])


splines = [Spline(u, B2, cpoints[ids]) for ids in iter_slices(3, len(cpoints))]

In [4]:
controls = {
    'cell': widgets.Checkbox(description="cell", value=True),
#     'Ka': widgets.FloatSlider(description="Ka", value=0.3, min=0, max=1.0, step=0.01, continuous_update=False),
    'Kw': widgets.FloatSlider(description="Kw", value=0.5, min=0, max=1.0, step=0.01, continuous_update=False),
    'Kc': widgets.FloatSlider(description="Kc", value=0.5, min=0, max=1.0, step=0.01, continuous_update=False),
    'Ke': widgets.FloatSlider(description="Ke", value=0.0, min=-1.0, max=1.0, step=0.01, continuous_update=False),
    'Kz': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False)
}
plot = k3d.plot(grid_visible=False)
plot += k3d.vectors(*make_grid((-1,1,1),(-1,1,1)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-0.5,0.5,1),(-0.5,0.5,1)), color=0x000000, use_head=False, line_width=0.01)

widgets.GridBox([widgets.VBox(list(controls.values())), plot], layout=widgets.Layout(grid_template_columns="auto auto"))

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


GridBox(children=(VBox(children=(Checkbox(value=True, description='cell'), FloatSlider(value=0.5, continuous_u…

In [5]:
plot_points1 = k3d.points(positions=[], color=palette[0], point_size=0.05)
plot += plot_points1
plot_lines1 = k3d.line(vertices=[], color=palette[1])
plot += plot_lines1
plot_lines2 = k3d.line(vertices=[], color=palette[2])
plot += plot_lines2

In [6]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    points0 = np.array([[c.subs(params) for c in p] for p in cpoints])
    spline0 = np.concatenate([eval_curve(spl.subs(params), u) for spl in splines])
    if params['cell']:
        spline1 = np.concatenate([spline0, np.flip(spline0,0) * np.array(vector(-1, 1, 1))])
        spline2 = spline1 + np.array(vector(0, 1.0, 0))
    else:
        spline1 = spline0
        spline2 = spline1 + np.array(vector(0, 1.0, 0))
    plot_points1.positions = points0.astype(np.float32)
    plot_lines1.vertices = spline1.astype(np.float32)
    plot_lines2.vertices = spline2.astype(np.float32) 
    

In [7]:
redraw()

In [44]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')

# Topology

Defining anchor or approximated points

Main hypotesis: shape can be determined based only on local neighbours.

## Hexagonal

From ["3D Modeling of Plain Weft Knitted Structures of Compressible Yarn", Kyosev et al, 2010] 

Anchors:
* $A^{NW}, A^{NE}$ — intersection points near the tip of a loop head
* $A^{SW}, A^{SE}$ — intersection points at sides of a loop head

Symmetrical variations $z+$ and $z-$ for above/below yarn.

Anchored splines:
* head: $south(A^{NW}), A^{SW}, A^{NW}, A^{NE}, A^{SE}, south(A^{NE})$
* w leg: $west(A^{SE}), A^{SW}, A^{NW}, north(A^{SW})$
* e leg: $east(A^{SW}), A^{SE}, A^{NE}, north(A^{SE})$

Cells preserve type of head knots, legs points are fully dependant

## Triangular

Anchors (and approximants):
* $A^{N}$ — tip of head
* $A^{W}$ — tip of w counter head
* $A^{E}$ — tip of e counter head

Anchored splines:
* head: $ \approx south(A^{N}), \approx A^{W}, A^{N}, \approx A^{E}, \approx south(A^{N})$
* w leg:  $\approx west(A^{N}), A^{W}, \approx A^{N}, \approx north(A^{W})$
* e leg:  $\approx east(A^{N}), A^{E}, \approx A^{N}, \approx north(A^{E})$


In [16]:
base4 = {
    'nw': np.array([-0.125, 0.25, -0.5]),
    'ne': np.array([+0.125, 0.25, -0.5]),
    'sw': np.array([-0.375, -0.25, +0.5]),
    'se': np.array([+0.375, -0.25, +0.5]),
}

def topology4(pattern, x, y):
    # generate points for splines inside single cell
    k = pattern[y][x]
    if k is None:
        return None
    ks = pattern[y+1][x]
    if ks is None:
        ks = k
    kn = pattern[y-1][x]
    if kn is None:
        kn = k
    kw = pattern[y][x-1]
    if kw is None:
        kw = k
    ke = pattern[y][x+1]
    if ke is None:
        ke = k
    
    orient = np.array([1, 1, k])
    bump = np.array([0, 0, k])
    
    head = {
            'nw': base4['nw'] * orient,
            'ne': base4['ne'] * orient,
            'sw': base4['sw'] * orient,
            'se': base4['se'] * orient,
#             'snw': base['nw'] - vector(0, 1, 0),
#             'sne': base['ne'] - vector(0, 1, 0),
    }
    
    if k != kn:
        head['nw'] -= bump * 0.5
        head['ne'] -= bump * 0.5
        head['sw'] -= bump * 0.5
        head['se'] -= bump * 0.5 
        
    if k != kw:
        head['nw'] += bump * 0.5
        head['sw'] += bump * 0.5

    if k != ke:
        head['ne'] += bump * 0.5
        head['se'] += bump * 0.5
    
    legs = {
            'nw': head['nw'] + bump,
            'ne': head['ne'] + bump,
            'sw': head['sw'] - bump,
            'se': head['se'] - bump,
#             'wse': base['se'] - flip - vector(1, 0, 0),
#             'nsw': base['sw'] - flip + vector(0, 1, 0),
#             'esw': base['sw'] - flip + vector(1, 0, 0),
#             'nse': base['se'] - flip + vector(0, 1, 0),
    }
    return {
        'h': head,
        'l': legs
    }

In [67]:
base3 = {
    'n': np.array([   0, 0.25,-0.5]),
    'w': np.array([-0.5,-0.25,+0.5]),
    'e': np.array([+0.5,-0.25,+0.5]),
}

def topology3(pattern, x, y):
    # generate points for splines inside single cell
    k = pattern[y][x]
    if k is None:
        return None
    ks = pattern[y+1][x]
    if ks is None:
        ks = k
    kn = pattern[y-1][x]
    if kn is None:
        kn = k
    kw = pattern[y][x-1]
    if kw is None:
        kw = k
    ke = pattern[y][x+1]
    if ke is None:
        ke = k
    
    orient = np.array([1, 1, k])
    flip = np.array([1, 1, -k])
    bump = np.array([0, 0, k])
    
    head = {
        'n': base3['n'] * orient,
        'w': base3['w'] * orient,
        'e': base3['e'] * orient,
#         'sn':
    }

    if k != kn:
#         head['n'] -= bump * 0.5
        head['w'] -= bump * 0.5
        head['e'] -= bump * 0.5 

    if k != kw:
        head['w'] += bump * 0.5

    if k != ke:
        head['e'] += bump * 0.5
  
    
    legs = {
            'n': head['n'] + bump,
            'w': head['w'] - bump,
            'e': head['e'] - bump,
        #   'wn'
        #   'en'
        #   'nw'
        #   'ne'
    }
    
    return {
        'h': head,
        'l': legs
    }    


In [7]:
chk = lambda: widgets.Checkbox(value=True, indent=False, layout=widgets.Layout(width='32px'))
brd = widgets.VBox([
    widgets.HBox([chk(), chk(), chk()]), # N
    widgets.HBox([chk(), chk(), chk()]), # M
    widgets.HBox([chk(), chk(), chk()])  # S
])

def read_pattern():
    return [[int(box.value)*2-1 for box in hbox.children] for hbox in brd.children]

controls = {
    'topo': widgets.RadioButtons(description="topo", options=('hexagonal', 'triangular')),
    'mode': widgets.RadioButtons(description="mode", options=('polygon', 'spline'))
}
plot = k3d.plot(grid_visible=False)

widgets.GridBox([widgets.VBox([brd, *controls.values()]), plot], layout=widgets.Layout(grid_template_columns="auto auto"))


GridBox(children=(VBox(children=(VBox(children=(HBox(children=(Checkbox(value=True, indent=False, layout=Layou…

In [8]:
plot.grid_visible=True

In [9]:
plot += k3d.vectors(*make_grid((-2,2,1),(-2,2,1)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-1.5,1.5,1),(-1.5,1.5,1)), color=0x000000, use_head=False, line_width=0.01)

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [10]:
plot_points = k3d.points(positions=[], shader='3d', opacity=0.5, point_size=0.05, color=palette[0])
plot += plot_points
plot_row1 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[1])
plot += plot_row1
plot_row2 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[2])
plot += plot_row2
plot_row3 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[3])
plot += plot_row3

In [11]:
south = np.array([0, -1, 0])

In [31]:
def make_course4(headrow, legsrow):
    adj = np.array([0.0625, 0, 0])    
    if legsrow is None:
        legsrow = headrow
    return np.concatenate([
        np.array([
            legsrow[x]['l']['sw'] + adj + south,
            legsrow[x]['l']['nw'] + adj + south,
            headrow[x]['h']['sw'] - adj,
            headrow[x]['h']['nw'] - adj,
            headrow[x]['h']['ne'] + adj,
            headrow[x]['h']['se'] + adj,
            legsrow[x]['l']['ne'] - adj + south,
            legsrow[x]['l']['se'] - adj + south,
        ]) + np.array([x, 0, 0])  for x in range(3)])    

In [61]:
def make_course3(headrow, legsrow):
    adj = np.array([0, 0.125, 0])
    if legsrow is None:
        legsrow = headrow
    return np.concatenate([
        np.array([
            legsrow[x]['l']['w'] - adj + south,
            legsrow[x]['l']['n'] - adj + south,
            headrow[x]['h']['w'] + adj,
            headrow[x]['h']['n'] + adj,
            headrow[x]['h']['e'] + adj,
            legsrow[x]['l']['n'] - adj + south,
#             legsrow[x]['l']['e'] - adj + south,            
        ]) + np.array([x, 0, 0])  for x in range(3)])    

In [25]:
def redraw():
    params = {k: w.value for k, w in controls.items()}
    pattern = read_pattern()
    pattern = [[None,None,None,None,None]] + [[None] + row + [None] for row in pattern] + [[None,None,None,None,None]]
    
    if params['topo'] == 'triangular':
        topology = topology3
        make_course = make_course3
    else:
        topology = topology4
        make_course = make_course4
        
    cells = [[topology(pattern, x, y) for x in range(1,4)] for y in range(1,4)]
    
    plot_points.positions = np.concatenate([
        np.concatenate([
            np.array(list(cells[y][x]['h'].values())) + np.array([-1+x, -1+y, 0])
            for x in range(3)
        ])
        for y in range(3)
    ]) * np.array([1, 1, 0])

    if params['mode'] == 'polygon':
        plot_row1.vertices = make_course(cells[0], cells[1]) * np.array([1, 1, .5]) + np.array([-1, +1, 0])
        plot_row2.vertices = make_course(cells[1], cells[2]) * np.array([1, 1, .5]) + np.array([-1,  0, 0])
        plot_row3.vertices = make_course(cells[2], None) * np.array([1, 1, .5]) + np.array([-1, -1, 0])
    else:
        plot_row1.vertices = eval_splines(B2, make_course(cells[0], cells[1]) * np.array([1, 1, .5]) + np.array([-1, +1, 0]))
        plot_row2.vertices = eval_splines(B2, make_course(cells[1], cells[2]) * np.array([1, 1, .5]) + np.array([-1,  0, 0]))
        plot_row3.vertices = eval_splines(B2, make_course(cells[2], None) * np.array([1, 1, .5]) + np.array([-1, -1, 0]))
    

In [44]:
redraw()

In [43]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')
for r in brd.children:
    for w in r.children:
        w.observe(lambda *args: redraw(), 'value')

# Geometry

### Parameters
* real scale to get correct thickness
* yarn thickness

### Manual control
* wale width
* course width

### Constraints

* symmetry at cell boundaries to align spline segments
* distance between splines at knot point > r
* distance between splines and cell edges > r